# Step 1 - Importing the necessary libraries for dataset loading, manipulation and building the model

In [33]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os

# Step -2 Preparing the dataset from the directories

In [34]:
# creating paths for the urls
train_dir = "../datasets/TrainingDataset/"
test_dir = "../datasets/TestingDataset/"
sample_image = cv2.imread(os.path.join(train_dir, "Covid/1-s2.0-S0140673620303706-fx1_lrg.jpg"))

# getting the sample image info
image_size = sample_image.shape
print(image_size[0])

image_size = (180, 180, 3)

1148


In [35]:
class PrepareData:
    def __init__(self, directory_path : str, image_size : tuple, batch_size : int) -> DataLoader:
        self.directory_path = directory_path
        self.image_size = image_size
        self.batch_size = batch_size

    def prepare_dataset(self):
        transform = transforms.Compose([
                transforms.Resize((image_size[0], image_size[1])),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
            ])
        
        dataset = datasets.ImageFolder(
            root=self.directory_path,
            transform=transform
        )

        loaded_data = DataLoader(
                dataset,
                batch_size=self.batch_size,
                shuffle=True,
                num_workers=4
            )
        
        return loaded_data

In [36]:
test_dataset_instance = PrepareData(directory_path=test_dir, batch_size=8, image_size=image_size)
train_dataset_instance = PrepareData(directory_path=train_dir, batch_size=8, image_size=image_size)

In [37]:
print("________Prepating the train dataset__________")
train_loader = train_dataset_instance.prepare_dataset()
print("Process completed successfully!")

print("\n")

print("________Prepating the test dataset__________")
test_loader = test_dataset_instance.prepare_dataset()
print("Process completed successfully!")

________Prepating the train dataset__________
Process completed successfully!


________Prepating the test dataset__________
Process completed successfully!


# Step - 3 Building the CNN Architecture

In [38]:
class CNN(nn.Module):
    def __init__(self, num_classes=2):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: (90, 90, 32)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: (45, 45, 64)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: (22, 22, 128)

        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: (11, 11, 256)

        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(256 * 11 * 11, 512)
        self.relu_fc1 = nn.ReLU()
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool1(self.relu1(self.bn1(self.conv1(x))))
        x = self.pool2(self.relu2(self.bn2(self.conv2(x))))
        x = self.pool3(self.relu3(self.bn3(self.conv3(x))))
        x = self.pool4(self.relu4(self.bn4(self.conv4(x))))
        
        x = x.view(x.size(0), -1)  # Flatten
        
        x = self.dropout(self.relu_fc1(self.fc1(x)))
        x = self.fc2(x)  # For binary classification
        
        return x

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device: ",device)

model = CNN(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Device:  cpu


# Step - 4 Training the Model

In [40]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/10], Loss: 2.0862
Epoch [2/10], Loss: 3.2926
Epoch [3/10], Loss: 1.2922
Epoch [4/10], Loss: 0.9662
Epoch [5/10], Loss: 0.9302
Epoch [6/10], Loss: 0.1949
Epoch [7/10], Loss: 0.0000
Epoch [8/10], Loss: 1.3325
Epoch [9/10], Loss: 4.1461
Epoch [10/10], Loss: 0.2482


# Step - 5 Evaluating the Model Performance

In [41]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on test set: {accuracy:.2f}%')

Accuracy on test set: 98.00%


# Step - 6 Model Deployment as a pytorch file

In [42]:
output_dir = "../models/covid_detector.pth"
torch.save(model.state_dict(), output_dir)

In [44]:
# checking the model's loading and evaluation

model.load_state_dict(torch.load(output_dir))
print(model.eval())
print("model is packaged and loaded perfectly...")

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu4): ReLU

# Testing of the Model's standalone performance

In [45]:
from PIL import Image

transform = transforms.Compose([
        transforms.Resize((180, 180)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
    ])

# Load and transform the image
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")  # Ensure RGB format
    image = transform(image)  # Apply transformations
    image = image.unsqueeze(0)  # Add batch dimension
    return image

In [46]:
# Define class labels
class_labels = ["Covid", "Normal"]

def predict_image(image_path):
    image = preprocess_image(image_path)

    # Forward pass through the model
    with torch.no_grad():
        output = model(image)
        predicted_class = torch.argmax(output, dim=1).item()

    return class_labels[predicted_class]

# Example usage
image_path = "../datasets/TestingDataset/Covid/1-s2.0-S0140673620303706-fx1_lrg.jpg"  # Replace with your image file
prediction = predict_image(image_path)
print(f"Prediction: {prediction}")


Prediction: Covid
